In [ ]:
# library
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gzip
import itertools

In [ ]:
!pip install biopython
from Bio import SeqIO

In [ ]:
# list to store strain names, accession numbers, and file names
strains = ['186','1D1609','1D1108','1D1460','15955','12D1','A6','CFBP5499','CFBP5877','CFBP6623','CFBP7129','CFBP6624','BIM B-1315G','FDAARGOS_1048','6N2']
accessions = ['GCF_002591665.3','GCF_002943835.1','GCF_003666425.1','GCF_003666445.1','GCF_003666465.1','GCF_003667905.1','GCF_003667925.1','GCF_005221325.1','GCF_005221345.1','GCF_005221385.1','GCF_005221405.1','GCF_005221425.1','GCF_014489975.1','GCF_016403145.1','GCF_017726655.1']
fasta_files = ['GCF_002591665.3_ASM259166v3_genomic.fna.gz','GCF_002943835.1_ASM294383v1_genomic.fna.gz','GCF_003666425.1_ASM366642v1_genomic.fna.gz','GCF_003666445.1_ASM366644v1_genomic.fna.gz','GCF_003666465.1_ASM366646v1_genomic.fna.gz','GCF_003667905.1_ASM366790v1_genomic.fna.gz','GCF_003667925.1_ASM366792v1_genomic.fna.gz','GCF_005221325.1_ASM522132v1_genomic.fna.gz','GCF_005221345.1_ASM522134v1_genomic.fna.gz','GCF_005221385.1_ASM522138v1_genomic.fna.gz','GCF_005221405.1_ASM522140v1_genomic.fna.gz','GCF_005221425.1_ASM522142v1_genomic.fna.gz','GCF_014489975.1_ASM1448997v1_genomic.fna.gz','GCF_016403145.1_ASM1640314v1_genomic.fna.gz','GCF_017726655.1_ASM1772665v1_genomic.fna.gz']

In [ ]:
# dictionary to store all seqs as str
dict_genomes = {}
for i in range(len(strains)):
    acc_id = strains[i]
    file_name = fasta_files[i]
    with gzip.open(file_name, "rt") as handle:
        records = list(SeqIO.parse(handle, "fasta"))
        dict_genomes[acc_id] = []
        for record in records:
            dict_genomes[acc_id].append(str(record.seq.upper()))

In [ ]:
for accession in dict_genomes.keys():
    for seq_str in dict_genomes[accession]:
        print(accession, len(seq_str))

## Function: get_kmer_mt
The multidimensional method

In [ ]:
def get_kmer_mt(seq_str, k=12, kmer_mt=np.zeros(shape=[4]*12)):
    seq_str = seq_str.replace('A', '0')
    seq_str = seq_str.replace('T', '1')
    seq_str = seq_str.replace('C', '2')
    seq_str = seq_str.replace('G', '3')
    i = 0
    while i+k <= len(seq_str):
        kmer_h = seq_str[i:i+k]
        coord = tuple([int(n) for n in kmer_h])
        kmer_mt[coord]+=1
        i+=1
    return kmer_mt

In [ ]:
k=12
genome = '15955'

kmer_mt = np.zeros(shape=[4]*k)
seq_str = dict_genomes[genome][0]
kmer_mt = get_kmer_mt(seq_str, k=k, kmer_mt=kmer_mt)

for contig in range(1, len(dict_genomes[genome])):
    seq_str = dict_genomes[genome][contig]
    kmer_mt = get_kmer_mt(seq_str, k=k, kmer_mt=kmer_mt)

# convert multidimensional matrix to sorted kmer table
data = {'coord':list(itertools.product([0,1,2,3], repeat=k)), 'count':kmer_mt.flatten()}
kmer_tab = pd.DataFrame(data)
kmer_tab = kmer_tab.sort_values('count', ignore_index=True, ascending=False)
kmer_tab

In [ ]:
kmer_tab1 = kmer_tab1.head(1000)
kmer_tab.plot(x='coord', y='count')

#### check function: get_kmer_mt

In [ ]:
kmer_h = ''.join([str(value) for value in kmer_tab.iloc[0,0]])
kmer_h = kmer_h.replace('0', 'A')
kmer_h = kmer_h.replace('1', 'T')
kmer_h = kmer_h.replace('2', 'C')
kmer_h = kmer_h.replace('3', 'G')
kmer_h

'GCCGCCGCCG'

In [ ]:
with gzip.open(fasta_files[4], "rt") as handle:
    records = list(SeqIO.parse(handle, "fasta"))
    for record in records:
        seq_h = record.seq.upper()
        print(seq_h.count_overlap(kmer_h))

111
77
22
2


## Function: get_kmer_tab
The hash algorithm

In [ ]:
def get_kmer_tab(seq_str, k=12):
    kmer_table = pd.DataFrame(columns = ['kmer','count'])
    i = 0
    while i+k <= len(seq_str):
        kmer_h = seq_str[i:i+k]
        if kmer_h in kmer_table['kmer'].tolist():
            kmer_table.loc[kmer_table['kmer']==kmer_h, 'count'] += 1
        else:
            kmer_table.loc[len(kmer_table.index)] = [kmer_h, 1]
        i+=1
    table.sort_values('count')
    return kmer_table

In [ ]:
seq_str = dict_genomes['186'][3]

In [ ]:
table = get_kmer_tab(seq_str, k=12)

In [ ]:
table

,kmer,count
0,TGTATCGATATC,1
1,GTATCGATATCG,1
2,TATCGATATCGT,1
3,ATCGATATCGTA,1
4,TCGATATCGTAT,1
...,...,...
64,ATCCGATATTGA,1
65,TCCGATATTGAC,1
66,CCGATATTGACT,1
67,CGATATTGACTA,1
